## Managing and Sharing Dynamic Compute Grafts
__________________
In this notebook we will provide an overview of the methods available to save, manage, and share your Dynamic Compute objects

In [ ]:
import descarteslabs.dynamic_compute as dc
from descarteslabs.dynamic_compute import ImageStack, Mosaic

In [ ]:
from uuid import uuid4

First lets create both an ImageStack and a Mosaic:

In [ ]:
image_stack = ImageStack.from_product_bands(
    "airbus:oneatlas:spot:v1",
    "red green blue",
    start_datetime="2021-01-01",
    end_datetime="2023-01-01",
)

mosaic = Mosaic.from_product_bands(
    "airbus:oneatlas:spot:v1",
    "red green blue",
    start_datetime="2021-01-01",
    end_datetime="2023-01-01",
)

## Saving Objects to Blobs
We can save Dynamic Compute objects using the `save_to_blob` function in the `Catalog` module of `Dynamic Compute`. The ID of the blob that is created is returned by the `save_to_blob` call. 

Note that the resulting storage type is __dyncomp/__.

In [ ]:
##Save a Mosaic with extra properties
mosaic_blob_id = dc.catalog.save_to_blob(
    mosaic,
    name=f"ExampleMosaic-{uuid4()}",
    description="A short description of this mosaic",
    extra_properties={"threshold": 0.1},
)

In [ ]:
##Save an ImageStack
stack_blob_id = dc.catalog.save_to_blob(
    image_stack,
    name=f"ExampleImageStack-{uuid4()}",
    description="A short description of this stack",
    extra_properties={"threshold": 0.1},
)

In [ ]:
##Saving a Band Ratio
red, green = mosaic.unpack_bands("red green")
ratio = (red - green) / (red + green)
ratio_blob_id = dc.catalog.save_to_blob(
    ratio, name=f"MySimpleRatio-{uuid4()}", description="simple ratio"
)
ratio_blob_id

## Loading Saved Objects
Now that we have saved each of them, we can used the `load_from_blob` function to retrieve the same state:

<b>*Important Notes*</b>
* Only load objects from sources you trust!
* Saved objects can only be loaded into an environment with the same Python version.

In [ ]:
loaded_stack = dc.catalog.load_from_blob(stack_blob_id)
all(loaded_stack == image_stack)

## Sharing Blobs
You can also share blobs with groups, users, emails, and organizations:

In [ ]:
dc.catalog.share_blob(
    mosaic_blob_id,
    emails=["email:john.daly@descarteslabs.com"],
    orgs=["org:pga-tour"],
    as_writers=True,
)

## Listing Dynamic Compute Blobs

If you forgot your ID can print blobs that you have access to:

In [ ]:
dc.catalog.print_blobs()

Or retrieve a list to iterate over:

In [ ]:
blobs = list(dc.catalog.find_blobs())
for blob in blobs:
    print(blob.name, blob.readers, blob.writers)

## Deleting Blobs
Lastly you can delete a Dynamic Compute blob:

In [ ]:
dc.catalog.delete_blob(mosaic_blob_id)
dc.catalog.delete_blob(stack_blob_id)

In [ ]:
dc.catalog.print_blobs()